In [ ]:
import json
import pandas as pd
import altair as alt
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

def df_from_cardano_node_logs(fp):
    lines = open(fp, "r").readlines()
    at_lines = [ json.loads(line) for line in lines if line.startswith('{"at') ]
    return pd.DataFrame.from_records(at_lines)

cardano_node_0_df = df_from_cardano_node_logs("data/cardano-node-0.log")
cardano_node_1_df = df_from_cardano_node_logs("data/cardano-node-1.log")

all_logs = pd.concat([cardano_node_0_df.assign(source="cardano-node-0"), cardano_node_1_df.assign(source="cardano-node-1")])

all_logs

In [ ]:
# SELECT * FROM all_logs WHERE sev = 'Critical'
all_logs[all_logs.sev == "Critical"]

In [ ]:
# SELECT DISTINCT_COUNT(ns), ARG_MAX(ns), ARG_MIN(ns), ARG_MAX(sev), ARG_MIN(sev) FROM all_logs GROUP BY source 
all_logs.groupby('source').agg({
    'ns': ['nunique', 'max', 'min'],
    'sev': ['max', 'min']
})

In [ ]:
all_logs.groupby('ns').count()

In [ ]:
latency_logs_df = all_logs[all_logs.ns.isin(["BlockFetch.Client.CompletedBlockFetch", "ChainSync.Client.DownloadedHeader"])]
latency_logs_df.loc[:, 'block'] = latency_logs_df.apply(lambda r: r.data['block'], axis=1)

In [ ]:
latency_logs_df.groupby(['block', 'source', 'ns']).agg({
    'at': ['count', 'min', 'max']
})